# How to use Semantic Kernel Memory with Qdrant vector store

### Setup Qdrant locally

To setup Qdrant locally follow the [Download and run](https://qdrant.tech/documentation/quick-start/#download-and-run) Qdrant steps.

```bash
docker pull qdrant/qdrant
```

Once the image is downloaded then run in a container.

```bash
docker run -p 6333:6333 -v $pwd/qdrant_storage:/qdrant/storage:z qdrant/qdrant
```

Set the .env file with the endpoint

QDRANT_ENDPOINT="http://localhost:6333"

In [1]:
#r "nuget: dotenv.net, 3.1.2"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.2

### Setup Semantic Kernel and Qdrant memeory

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-beta1"
#r "nuget: Newtonsoft.Json, 13.0.3"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Text;
using System.IO;
using System.Text.Json;
using Microsoft.SemanticKernel.Services;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;
using Microsoft.SemanticKernel.Diagnostics;
using Microsoft.SemanticKernel.Http;


var memoryStorage = new QdrantMemoryStore(endpoint: env["QDRANT_ENDPOINT"], vectorSize: 1536);
var textEmbeddingGenerator = new AzureTextEmbeddingGeneration(
            modelId: "text-embedding-ada-002",
            endpoint: env["AZURE_OPENAI_ENDPOINT"],
            apiKey: env["AZURE_OPENAI_API_KEY"]);

var memory = new SemanticTextMemory(memoryStorage, textEmbeddingGenerator);


var kernel = Kernel.Builder
        
    // Use Azure OpenAI for Embeddings (model: text-embedding-ada-002)
    .WithAzureTextEmbeddingGenerationService(
        deploymentName: "text-embedding-ada-002",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])

    // Use Azure OpenAI for Semantic Functions (model = gpt-35-turbo)
    .WithAzureChatCompletionService(
        deploymentName: "gpt-35-turbo",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])
        
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta1 Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-beta1 Newtonsoft.Json, 13.0.3

### Populate the Qdrant vector store with data

In [3]:
using System.IO;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;


string json = File.ReadAllText("data/text-sample.json");
dynamic data = JsonConvert.DeserializeObject(json);

// Sample documents are small enough so there is no need for chunking. Otherwise, it is recommended to chunk data before storing. 

int i = 0;
foreach (JObject obj in data)
{
    var key = await memory.SaveInformationAsync(
        "skidx1",
        obj.ToString(),
        $"text-sample-{i}",
        $"Dataset: text-sample Chunk: {i}",
        i.ToString());

    i++;
}

### Get the intent of the question being asked

In [4]:
var question = "What services are best for asynchornous communications?";

In [5]:
// Intent detection

using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

const string IntentFunctionDefinition = @"
Bot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 5 words or less: ";

var intent = kernel.CreateSemanticFunction(IntentFunctionDefinition, requestSettings: new OpenAIRequestSettings() { MaxTokens = 500, Temperature = 0.0 });

var result = await kernel.RunAsync(intent, new()
    {
        ["input"] = question
    });

var resultIntent = result.GetValue<string>();
Console.WriteLine($"{resultIntent}");

Asynchronous communication services recommendation.


### Recall from memory (Sanity Check)

In [6]:
// Search from memory for sanity check

using Microsoft.SemanticKernel.Orchestration;

// Import the TextMemoryPlugin into the Kernel for other functions
var memoryPlugin = new TextMemoryPlugin(memory);
var memoryFunctions = kernel.ImportFunctions(memoryPlugin);

var result = await kernel.RunAsync(memoryFunctions["Recall"], new()
    {
        [TextMemoryPlugin.CollectionParam] = "skidx1",
        [TextMemoryPlugin.LimitParam] = "3",
        //[TextMemoryPlugin.RelevanceParam] = "0.79",
        ["input"] = resultIntent
    });


foreach (var functionResult in result.FunctionResults)
{
    Console.Write(functionResult.GetValue<string>());
}


["{\r\n  \u0022id\u0022: \u002228\u0022,\r\n  \u0022title\u0022: \u0022Azure Service Bus\u0022,\r\n  \u0022content\u0022: \u0022Azure Service Bus is a fully managed, enterprise-grade messaging service that enables you to build reliable and scalable applications. It provides features like message queuing, publish-subscribe, and dead-lettering. Service Bus supports various messaging patterns, including point-to-point, broadcast, and request-reply. You can use Service Bus to integrate your applications and services, decouple your system components, and handle asynchronous communication. It also integrates with other Azure services, such as Azure Functions and Azure Logic Apps.\u0022,\r\n  \u0022category\u0022: \u0022Integration\u0022\r\n}","{\r\n  \u0022id\u0022: \u002264\u0022,\r\n  \u0022title\u0022: \u0022Azure SignalR Service\u0022,\r\n  \u0022content\u0022: \u0022Azure SignalR Service is a fully managed, real-time messaging service that enables you to build and scale real-time web ap

### Search and Summarize the result

In [7]:
// Search & Summarize

using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

const string RecallFunctionDefinition = @"
Considering these facts

Facts: {{recall '{{$intent}}'}}

Question: {{$input}}

Provide a concise answer ('Answer: ') and a separate explanation ('Explanation: '), in two lines.
";

var aboutMeOracle = kernel.CreateSemanticFunction(RecallFunctionDefinition, requestSettings: new OpenAIRequestSettings() { MaxTokens = 1024, Temperature = 0.5 });

result = await kernel.RunAsync(aboutMeOracle, new()
    {
        [TextMemoryPlugin.CollectionParam] = "skidx1",
        [TextMemoryPlugin.LimitParam] = "3",
        //[TextMemoryPlugin.RelevanceParam] = "0.79",
        ["input"] = question,
        ["intent"] = resultIntent
    });


Console.WriteLine($"Original Question: {question}");
Console.WriteLine($"Optimized Question: {resultIntent}");
Console.WriteLine("---");
Console.WriteLine($"{result.GetValue<string>()}");

Original Question: What services are best for asynchornous communications?
Optimized Question: Asynchronous communication services recommendation.
---
Answer: Azure Logic Apps and Azure Bot Service are best for asynchronous communications.

Explanation: Azure Logic Apps allows you to create and run workflows that integrate with various services, making it suitable for asynchronous communication. Azure Bot Service enables you to build and manage intelligent chatbots, which can handle asynchronous conversations with users.
